In [ ]:
pip install pmdarima

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 9.3 MB/s eta 0:00:00


## ARIMA

In [ ]:
from statsmodels.tsa.arima.model import ARIMA
from datetime import timedelta
import pandas as pd
from pmdarima.arima import auto_arima
import numpy as np
from scipy.stats import norm
from statsmodels.tsa.holtwinters import ExponentialSmoothing
def arima(test_week):
    quantiles = np.array([0.01, 0.025] + list(np.arange(0.05, 0.95+0.05, 0.05)) + [0.975, 0.99])
    data_file = '/content/drive/MyDrive/Covid forecasting/Data/smoothed_rate_withcovid.csv'

    prediction_file = '/content/drive/MyDrive/Covid forecasting/Result/state level/'

    quantiles = np.array([0.01, 0.025] + list(np.arange(0.05, 0.95+0.05, 0.05)) + [0.975, 0.99])


    df = pd.read_csv(data_file)
    df['fips'] = df['fips'].astype(str).str.zfill(2)
    df['Week_end'] = pd.to_datetime(df['Week_end'])
    df = df.drop(['PRCP_mean', 'SNOW_mean', 'TMAXDELTA', 'TMINDELTA','s05_value','scontrol_value'], axis=1)
    #df_train = df[(df['Week_end'] < pd.to_datetime(test_week)) & (df['Week_end'] > pd.to_datetime('2022-08-01'))]
    df_train = df[(df['Week_end'] < pd.to_datetime(test_week))]
    df_train['covid_rate'] = np.log(df_train['covid_rate'] + 1)

    test_states = df_train['fips'].unique()

    quantile_result = pd.DataFrame(columns=['fips', 'Week_end', 'horizon', 'Quantile', 'Value'])

    # Accumulate entries in a list
    results = []

    for fips in test_states:
        df_state_train = df_train[df_train['fips'] == fips].set_index('Week_end').copy()

        #model = auto_arima(df_state_train['covid_rate'], seasonal=False, stepwise=True,
          #                suppress_warnings=True, error_action="ignore", trace=True, max_order=None, test="adf")

        #model = ARIMA(df_state_train['covid_rate'], order=model.order)

        #model = ExponentialSmoothing(df_state_train['covid_rate'], trend='add', seasonal='add', seasonal_periods=52)
        model = ExponentialSmoothing(df_state_train['covid_rate'], trend=None, seasonal=None)
        model_fit = model.fit()


        forecast = model_fit.forecast(steps=4)
        forecast_values = np.array(forecast)
        residuals = model_fit.resid
        residual_std = np.std(residuals)

        for horizon in range(0, 4):
            scaled_std = residual_std * np.sqrt(horizon + 1)
            quantile_forecasts = norm.ppf(quantiles, loc=forecast_values[horizon], scale=scaled_std)
            quantile_forecasts = np.exp(quantile_forecasts) - 1

            for q, quantile_forecast in zip(quantiles, quantile_forecasts):
                results.append({
                    'fips': fips,
                    'Week_end': pd.to_datetime(test_week),
                    'horizon': horizon,
                    'Quantile': q,
                    'Value': max(0, quantile_forecast)
                })


    quantile_result = pd.concat([quantile_result, pd.DataFrame(results)], ignore_index=True)
    pop = pd.read_csv('/content/drive/MyDrive/Flu Forecasting/code/population.csv')

    pop['fips'] = pop['fips'].astype(str).str.zfill(2)
    df = quantile_result.merge(pop, on='fips', how='inner')

    df['Value'] = (df['Value'] * df['population'] / 100000).round().astype(int)
    df.drop(columns=['population'], inplace=True)
    df['Value'][df['Value'] < 0] = 0
    quantile_result1 = df.copy()

    quantile_result1.rename(columns={
        'fips': 'location',
        'Week_end': 'reference_date',
        'horizon': 'horizon',
        'Quantile': 'output_type_id',
        'Value': 'value'
    }, inplace=True)
    quantile_result1['value'][quantile_result1['value'] < 0] = 0

    quantile_result1['target'] = 'wk inc flu hosp'

    quantile_result1['horizon'] = quantile_result['horizon'].astype(int)  # Convert 'horizon' to integer if it's not already

    quantile_result1['reference_date'] = pd.to_datetime(quantile_result1['reference_date'])
    quantile_result1['target_end_date'] = quantile_result1['reference_date'] + quantile_result1['horizon'] * pd.to_timedelta(7, unit='D')

    quantile_result1['output_type'] = 'quantile'
    quantile_result1['target_end_date'] = pd.to_datetime(quantile_result1['target_end_date'])
    quantile_result1=quantile_result1[['reference_date', 'target', 'horizon', 'target_end_date', 'location', 'output_type', 'output_type_id', 'value']]
    quantile_result1.to_csv(prediction_file + test_week + '_quantileresultexp1.csv',index = False)

In [ ]:
from datetime import datetime, timedelta
start_date = datetime.strptime('2023-10-07', '%Y-%m-%d')
end_date = datetime.strptime('2024-02-24', '%Y-%m-%d')
current_date = start_date
dates_list = []

while current_date <= end_date:
    dates_list.append(current_date.strftime('%Y-%m-%d'))
    current_date += timedelta(days=7)

test_weeks = dates_list

for test_week in test_weeks:
    arima(test_week)

Streaming output truncated to the last 5000 lines.
/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:836: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:836: FutureWarning: No supported index is available. In the next version, calling this method in a model without a supported index will result in an exception.
  return get_prediction_index(
/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:836: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start

In [ ]:
test_weeks

['2023-10-07',
 '2023-10-14',
 '2023-10-21',
 '2023-10-28',
 '2023-11-04',
 '2023-11-11',
 '2023-11-18',
 '2023-11-25',
 '2023-12-02',
 '2023-12-09',
 '2023-12-16',
 '2023-12-23',
 '2023-12-30',
 '2024-01-06',
 '2024-01-13',
 '2024-01-20',
 '2024-01-27',
 '2024-02-03',
 '2024-02-10',
 '2024-02-17',
 '2024-02-24']

### Plot

In [ ]:
import pandas as pd
quantile_result = pd.read_csv('/content/drive/MyDrive/Covid forecasting/Result/state level/2024-01-13_quantileresultexp.csv')
quantile_result

,reference_date,target,horizon,target_end_date,location,output_type,output_type_id,value
0,2024-01-13,wk inc flu hosp,0,2024-01-13,1,quantile,0.010,416
1,2024-01-13,wk inc flu hosp,0,2024-01-13,1,quantile,0.025,441
2,2024-01-13,wk inc flu hosp,0,2024-01-13,1,quantile,0.050,464
3,2024-01-13,wk inc flu hosp,0,2024-01-13,1,quantile,0.100,492
4,2024-01-13,wk inc flu hosp,0,2024-01-13,1,quantile,0.150,512
...,...,...,...,...,...,...,...,...
4595,2024-01-13,wk inc flu hosp,3,2024-02-03,56,quantile,0.850,69
4596,2024-01-13,wk inc flu hosp,3,2024-02-03,56,quantile,0.900,77
4597,2024-01-13,wk inc flu hosp,3,2024-02-03,56,quantile,0.950,90
4598,2024-01-13,wk inc flu hosp,3,2024-02-03,56,quantile,0.975,104


In [ ]:
hist = pd.read_csv('/content/drive/MyDrive/Covid forecasting/Data/admission_withcovid.csv')
#hist = pd.read_csv('/Users/shawn/Flu Forecasting/code/trainingdata.csv')

import pandas as pd
import matplotlib.pyplot as plt

quantile_result2 = quantile_result[quantile_result['reference_date'] == '2024-01-13']
quantile_result2
hist['Week_end'] = pd.to_datetime(hist['Week_end'])
#hist['fips'] =hist['fips'].astype(str).str.zfill(2)
quantile_result2['target_end_date'] = pd.to_datetime(quantile_result2['target_end_date'])

quantiles = [0.025, 0.5, 0.975]
quantile_data = quantile_result2[quantile_result2['output_type_id'].isin(quantiles)]

filtered_hist = hist[(hist['Week_end'] >= '2022-09-01')]
filtered_quantile_data = quantile_data[(quantile_data['target_end_date'] >= '2023-09-01')]

unique_states = quantile_result2['location'].unique()

for state in unique_states:
    state_hist_data = filtered_hist[filtered_hist['fips'] == state]
    state_quantile_data = filtered_quantile_data[filtered_quantile_data['location'] == state]

    plt.figure(figsize=(12, 6))

    plt.plot(state_hist_data['Week_end'], state_hist_data['covid_admissions'], marker='o', linestyle='-', label=f'{state} - Historical')

    for q in quantiles:

        q_data = state_quantile_data[state_quantile_data['output_type_id'] == q]
        plt.plot(q_data['target_end_date'], q_data['value'], marker='', linestyle='--', label=f'{state} - Quantile {q}')

        # Fill the area between 0.025 and 0.975 quantiles with color
        if q == 0.025:
            lower_q_data = q_data
        else:
            upper_q_data = q_data

    plt.fill_between(lower_q_data['target_end_date'], lower_q_data['value'], upper_q_data['value'], color='gray', alpha=0.5)

    plt.title(f'Total Admissions for {state} (Oct 1, 2023 - Dec 02, 2023)')
    plt.xlabel('End Date')
    plt.ylabel('Total Admissions')
    plt.xlim(state_hist_data['Week_end'].min(), state_quantile_data['target_end_date'].max())
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.show()


Output hidden; open in https://colab.research.google.com to view.

In [ ]:
state_hist_data

,fips,Week_end,PRCP_mean,SNOW_mean,TMAXDELTA,TMINDELTA,COVID19-related_value,COVID19_value,influenza_value,s01_value,...,s05_value,s06_value,scontrol_value,flu_difference,smoothed_flu_admission,flu_rate,state,covid_admissions,covid_difference,smoothed_covid_admission
87,1,2022-09-03,295.857143,0.0,65.333333,19.666667,1.443846,1.021598,0.029791,1.883759,...,0.092245,1.013923,4.256395,4.0,18.000000,0.453265,AL,680.0,-28.0,705.666667
88,1,2022-09-10,84.000000,0.0,54.888889,25.333333,1.241228,0.867409,0.035691,1.904065,...,0.085624,0.988776,4.228803,-13.0,17.333333,0.197072,AL,559.0,-121.0,649.000000
89,1,2022-09-17,106.142857,0.0,37.666667,47.000000,0.991788,0.650948,0.042715,1.874694,...,0.073946,0.946712,4.221891,7.0,16.666667,0.335022,AL,443.0,-116.0,560.666667
90,1,2022-09-24,12.714286,0.0,43.222222,58.000000,0.717941,0.427070,0.044854,1.872908,...,0.068254,0.930635,4.222163,2.0,15.333333,0.374436,AL,323.0,-120.0,441.666667
91,1,2022-10-01,4.428571,0.0,50.000000,75.777778,0.591945,0.320884,0.071355,1.934745,...,0.063787,0.941270,4.196558,4.0,19.666667,0.453265,AL,289.0,-34.0,351.666667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
165,1,2024-03-09,0.000000,0.0,76.805556,79.944444,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,-7.0,84.000000,1.537159,AL,305.0,-57.0,373.333333
166,1,2024-03-16,182.000000,0.0,84.444444,76.777778,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,3.0,81.333333,1.596281,AL,251.0,-54.0,306.000000
167,1,2024-03-23,295.285714,0.0,121.000000,121.888889,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,-22.0,72.666667,1.162723,AL,198.0,-53.0,251.333333
168,1,2024-03-30,107.714286,0.0,108.000000,121.555556,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,3.0,67.333333,1.221844,AL,186.0,-12.0,211.666667
